In [1]:
import pandas as pd
import requests
import time
from tqdm import tqdm

# Step 1: Read your CSV file
df = pd.read_csv('/data/ep924610/project_nb/paper_code/heatmap_results/skin_results.csv')
gene_symbols = df['Gene'].unique().tolist()

# Step 2: Convert gene symbols to UniProt IDs
def get_uniprot_id(gene_symbol):
    """Convert a gene symbol to UniProt ID using UniProt API"""
    url = f"https://rest.uniprot.org/uniprotkb/search?query=gene:{gene_symbol}+AND+reviewed:true+AND+organism_id:9606"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            return data['results'][0]['primaryAccession']
    return None

# Create a dictionary mapping gene symbols to UniProt IDs
print("Converting gene symbols to UniProt IDs...")
uniprot_dict = {}
for gene in tqdm(gene_symbols):
    uniprot_id = get_uniprot_id(gene)
    if uniprot_id:
        uniprot_dict[gene] = uniprot_id
    time.sleep(0.5)  # Avoid overwhelming the API

Converting gene symbols to UniProt IDs...


100%|███████████████████████████████████████████████████████████████████████████| 10837/10837 [2:11:14<00:00,  1.38it/s]


In [2]:
def get_protein_expression(uniprot_id):
    """Get protein expression data for a given UniProt ID"""
    base_url = "https://www.proteomicsdb.org/proteomicsdb/logic/api/proteinexpression.xsodata"
    
    # Create the param string with proper URL encoding
    query = f"/InputParams(PROTEINFILTER='{uniprot_id}',MS_LEVEL=1,TISSUE_ID_SELECTION='',TISSUE_CATEGORY_SELECTION='tissue;fluid',SCOPE_SELECTION=1,GROUP_BY_TISSUE=1,CALCULATION_METHOD=0,EXP_ID=-1)/Results"
    url = base_url + query + "?$select=UNIQUE_IDENTIFIER,TISSUE_ID,TISSUE_NAME,NORMALIZED_INTENSITY&$format=json"
    
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    return None

# Step 4: Calculate average expression scores
vital_tissues = ['brain', 'heart', 'lung']
results = []

print("Fetching protein expression data...")
for gene, uniprot_id in tqdm(uniprot_dict.items()):
    expression_data = get_protein_expression(uniprot_id)
    if not expression_data:
        continue
        
    vital_expressions = []
    non_vital_expressions = []
    
    for entry in expression_data.get('d', {}).get('results', []):
        tissue_name = entry.get('TISSUE_NAME', '').lower()
        
        # Convert expression value to float to avoid type issues
        try:
            expression = float(entry.get('NORMALIZED_INTENSITY', 0))
        except (TypeError, ValueError):
            # If conversion fails, use 0 as default
            expression = 0.0
        
        if any(vital in tissue_name for vital in vital_tissues):
            vital_expressions.append(expression)
        else:
            non_vital_expressions.append(expression)
    
    avg_vital = sum(vital_expressions) / len(vital_expressions) if vital_expressions else 0
    avg_non_vital = sum(non_vital_expressions) / len(non_vital_expressions) if non_vital_expressions else 0
    
    results.append({
        'Gene': gene,
        'UniProt_ID': uniprot_id,
        'Avg_Vital_Expression': avg_vital,
        'Avg_NonVital_Expression': avg_non_vital,
        'Vital_NonVital_Ratio': avg_vital / avg_non_vital if avg_non_vital > 0 else 0
    })
    
    time.sleep(1)  # Avoid overwhelming the API

# Create a DataFrame with the results
results_df = pd.DataFrame(results)
results_df.to_csv('/data/ep924610/project_nb/paper_code/protein_expression/protein_expression_results.csv', index=False)
print("Done! Results saved to protein_expression_results.csv")

Fetching protein expression data...


100%|███████████████████████████████████████████████████████████████████████████| 10667/10667 [5:37:16<00:00,  1.90s/it]

Done! Results saved to protein_expression_results.csv


In [3]:
vital_expressions

[7.240338444260151, 7.114794495283671, 7.529530422277909]

In [4]:
expression_data

{'d': {'results': [{'__metadata': {'type': 'proteomicsdb.logic.api.proteinexpression.CA_EXPRESSION_PROFILE_APIType',
     'uri': "http://www.proteomicsdb.org/proteomicsdb/logic/api/proteinexpression.xsodata/CA_EXPRESSION_PROFILE_API('148191442058377231')"},
    'UNIQUE_IDENTIFIER': 'P62328',
    'TISSUE_ID': 'BTO:0000037',
    'TISSUE_NAME': 'renal cell carcinoma cell',
    'NORMALIZED_INTENSITY': '8.020890635771687'},
   {'__metadata': {'type': 'proteomicsdb.logic.api.proteinexpression.CA_EXPRESSION_PROFILE_APIType',
     'uri': "http://www.proteomicsdb.org/proteomicsdb/logic/api/proteinexpression.xsodata/CA_EXPRESSION_PROFILE_API('148191442058377232')"},
    'UNIQUE_IDENTIFIER': 'P62328',
    'TISSUE_ID': 'BTO:0000047',
    'TISSUE_NAME': 'adrenal gland',
    'NORMALIZED_INTENSITY': '6.634121567667114'},
   {'__metadata': {'type': 'proteomicsdb.logic.api.proteinexpression.CA_EXPRESSION_PROFILE_APIType',
     'uri': "http://www.proteomicsdb.org/proteomicsdb/logic/api/proteinexpression

In [6]:
# Sort the results by vital tissue expression (from highest to lowest)
sorted_results = results_df.sort_values(by='Avg_Vital_Expression', ascending=True)

# Print the top 10 genes with highest expression in vital tissues
print("\nTop 10 genes with highest expression in vital tissues:")
top_10 = sorted_results.head(10)
print(top_10[['Gene', 'UniProt_ID', 'Avg_Vital_Expression', 'Avg_NonVital_Expression', 'Vital_NonVital_Ratio']])


Top 10 genes with highest expression in vital tissues:
         Gene UniProt_ID  Avg_Vital_Expression  Avg_NonVital_Expression  \
5814     LCOR     Q96JN0                   0.0                 3.636068   
4574   ZNF461     Q8TAF7                   0.0                 0.000000   
7090     IL24     Q13007                   0.0                 3.550842   
9529    SMAD7     O15105                   0.0                 0.000000   
7088     KLF7     O75840                   0.0                 4.832241   
7939   ZBTB46     Q86UZ6                   0.0                 3.630000   
7087    CNTD1     Q8N815                   0.0                 0.000000   
7084   ARRDC4     Q8NCT1                   0.0                 0.000000   
7079   ZNF630     Q2M218                   0.0                 3.251181   
7078  SLC35E4     Q6ICL7                   0.0                 4.265198   

      Vital_NonVital_Ratio  
5814                   0.0  
4574                   0.0  
7090                   0.0  
95

In [7]:
# Sort the results by vital tissue expression (from highest to lowest)
sorted_results = results_df.sort_values(by='Avg_Vital_Expression', ascending=True)

# Reset index to have row numbers from 0
sorted_results = sorted_results.reset_index(drop=True)

# Find the rank of GPNMB
gpnmb_row = sorted_results[sorted_results['Gene'] == 'GPNMB']
if not gpnmb_row.empty:
    gpnmb_index = gpnmb_row.index[0]
    gpnmb_rank = gpnmb_index + 1  # +1 because index is 0-based
    
    print(f"\nGPNMB rank when sorted by vital expression: {gpnmb_rank} out of {len(sorted_results)}")
    print("\nGPNMB details:")
    print(gpnmb_row[['Gene', 'UniProt_ID', 'Avg_Vital_Expression', 'Avg_NonVital_Expression', 'Vital_NonVital_Ratio']])
else:
    print("\nGPNMB not found in the results")


GPNMB rank when sorted by vital expression: 5785 out of 10667

GPNMB details:
       Gene UniProt_ID  Avg_Vital_Expression  Avg_NonVital_Expression  \
5784  GPNMB     Q14956              4.316531                 4.717716   

      Vital_NonVital_Ratio  
5784              0.914962  


In [8]:
gpnmb_row = sorted_results[sorted_results['Gene'] == 'ERBB3']
if not gpnmb_row.empty:
    gpnmb_index = gpnmb_row.index[0]
    gpnmb_rank = gpnmb_index + 1  # +1 because index is 0-based
    
    print(f"\nGPNMB rank when sorted by vital expression: {gpnmb_rank} out of {len(sorted_results)}")
    print("\nGPNMB details:")
    print(gpnmb_row[['Gene', 'UniProt_ID', 'Avg_Vital_Expression', 'Avg_NonVital_Expression', 'Vital_NonVital_Ratio']])
else:
    print("\nGPNMB not found in the results")


GPNMB rank when sorted by vital expression: 2621 out of 10667

GPNMB details:
       Gene UniProt_ID  Avg_Vital_Expression  Avg_NonVital_Expression  \
2620  ERBB3     P21860              3.507384                 3.756212   

      Vital_NonVital_Ratio  
2620              0.933756  


In [9]:
def get_protein_expression(uniprot_id):
    """Get protein expression data for a given UniProt ID"""
    base_url = "https://www.proteomicsdb.org/proteomicsdb/logic/api/proteinexpression.xsodata"
    
    # Create the param string with proper URL encoding
    query = f"/InputParams(PROTEINFILTER='{uniprot_id}',MS_LEVEL=1,TISSUE_ID_SELECTION='',TISSUE_CATEGORY_SELECTION='tissue;fluid',SCOPE_SELECTION=1,GROUP_BY_TISSUE=1,CALCULATION_METHOD=0,EXP_ID=-1)/Results"
    url = base_url + query + "?$select=UNIQUE_IDENTIFIER,TISSUE_ID,TISSUE_NAME,NORMALIZED_INTENSITY&$format=json"
    
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    return None

# Step 4: Calculate average expression scores and store individual tissue data
vital_tissues = ['brain', 'heart', 'lung']
results = []
all_tissue_names = set()  # To keep track of all tissues encountered

# First pass: collect all unique tissue names
print("First pass: collecting all tissue names...")
for gene, uniprot_id in tqdm(list(uniprot_dict.items())[:10]):  # Sample with first 10 genes to discover tissue names
    expression_data = get_protein_expression(uniprot_id)
    if not expression_data:
        continue
        
    for entry in expression_data.get('d', {}).get('results', []):
        tissue_name = entry.get('TISSUE_NAME', '').lower().replace(' ', '_').replace('-', '_')
        all_tissue_names.add(tissue_name)
    
    time.sleep(0.01)  # Avoid overwhelming the API

all_tissue_names = sorted(list(all_tissue_names))
print(f"Found {len(all_tissue_names)} unique tissues")

# Second pass: collect expression data for all genes
print("Second pass: fetching protein expression data for all genes...")
for gene, uniprot_id in tqdm(uniprot_dict.items()):
    expression_data = get_protein_expression(uniprot_id)
    if not expression_data:
        continue
        
    vital_expressions = []
    non_vital_expressions = []
    
    # Initialize the result dictionary with gene info and zeros for all tissues
    result = {
        'Gene': gene,
        'UniProt_ID': uniprot_id
    }
    
    # Add a column for each tissue with default value 0
    for tissue in all_tissue_names:
        result[f'tissue_{tissue}'] = 0.0
    
    # Fill in the actual expression values
    for entry in expression_data.get('d', {}).get('results', []):
        tissue_name = entry.get('TISSUE_NAME', '').lower()
        tissue_key = tissue_name.replace(' ', '_').replace('-', '_')
        
        # Convert expression value to float
        try:
            expression = float(entry.get('NORMALIZED_INTENSITY', 0))
        except (TypeError, ValueError):
            expression = 0.0
        
        # Store the expression value in the tissue-specific column
        result[f'tissue_{tissue_key}'] = expression
        
        # Also keep track for vital vs non-vital calculation
        if any(vital in tissue_name for vital in vital_tissues):
            vital_expressions.append(expression)
        else:
            non_vital_expressions.append(expression)
    
    # Calculate averages
    avg_vital = sum(vital_expressions) / len(vital_expressions) if vital_expressions else 0
    avg_non_vital = sum(non_vital_expressions) / len(non_vital_expressions) if non_vital_expressions else 0
    
    # Add the summary columns
    result['Avg_Vital_Expression'] = avg_vital
    result['Avg_NonVital_Expression'] = avg_non_vital
    result['Vital_NonVital_Ratio'] = avg_vital / avg_non_vital if avg_non_vital > 0 else 0
    
    results.append(result)
    
    time.sleep(0.01)  # Avoid overwhelming the API

# Create a DataFrame with the results
results_df = pd.DataFrame(results)
results_df.to_csv('/data/ep924610/project_nb/paper_code/protein_expression/protein_expression_results_with_tissues.csv', index=False)
print("Done! Results saved to protein_expression_results_with_tissues.csv")

# Example of how to find genes with high expression in a specific tissue
def find_genes_by_tissue(df, tissue_name, min_expression=7.0):
    """Find genes with expression above threshold in a specific tissue"""
    tissue_col = f'tissue_{tissue_name.lower().replace(" ", "_").replace("-", "_")}'
    if tissue_col not in df.columns:
        print(f"Tissue {tissue_name} not found. Available tissues: {[col.replace('tissue_', '') for col in df.columns if col.startswith('tissue_')]}")
        return pd.DataFrame()
    
    return df[df[tissue_col] >= min_expression].sort_values(by=tissue_col, ascending=False)

# Example usage:
# high_brain_expression = find_genes_by_tissue(results_df, 'brain', min_expression=8.0)
# print(high_brain_expression[['Gene', 'UniProt_ID', 'tissue_brain']].head(10))

First pass: collecting all tissue names...


100%|███████████████████████████████████████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.25it/s]


Found 69 unique tissues
Second pass: fetching protein expression data for all genes...


100%|███████████████████████████████████████████████████████████████████████████| 10667/10667 [2:49:47<00:00,  1.05it/s]


Done! Results saved to protein_expression_results_with_tissues.csv


In [1]:
import pandas as pd
import requests
import time
from tqdm import tqdm


In [2]:
results_df = pd.read_csv("/data/ep924610/project_nb/paper_code/protein_expression/protein_expression_results_with_tissues.csv")

In [6]:
results_df[["Gene","Avg_Vital_Expression", "Avg_NonVital_Expression"]]

,Gene,Avg_Vital_Expression
0,GPNMB,4.316531
1,FMN1,3.398871
2,TMEM117,3.585851
3,GPM6B,5.110041
4,SGCD,5.008843
...,...,...
10662,SRGN,4.677084
10663,ZFP36,3.904304
10664,TNFAIP3,4.082586
10665,NFKBIA,4.306088


In [8]:
results_df[["Gene","Avg_Vital_Expression", "Avg_NonVital_Expression"]].sort_values(by="Avg_Vital_Expression", ascending=True)

,Gene,Avg_Vital_Expression,Avg_NonVital_Expression
5814,LCOR,0.000000,3.636068
4574,ZNF461,0.000000,0.000000
7090,IL24,0.000000,3.550842
9529,SMAD7,0.000000,0.000000
7088,KLF7,0.000000,4.832241
...,...,...,...
9705,YWHAZ,7.327124,7.159622
10555,TPI1,7.341830,6.915559
10540,TUBB4B,7.367555,7.202583
10457,TUBB,7.386673,7.088919
